<a href="https://colab.research.google.com/github/ram2311/SIMILAR-RECIPE-PREDICTION-USING-BLAST/blob/main/SIMILAR-RECIPE-PREDICTION-USING-BLAST_Project_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similar Recipe Prediction Using  BLAST and Recipe Sequences

In [ ]:
import json
import csv

### All required datasets are imported 
### All the data is taken from Recipedb

In [ ]:
#B L RAMA KRISHNA_18BIT0175
with open("C:/Users/varad/Desktop/CGAS_project/recipe_instruc.json") as jsonfile:
    recipe_instruc_main=json.load(jsonfile)

recipeid_instruc={}
for i in recipe_instruc_main:
    recipeid_instruc[int(i['Recipe_id'])]=i['steps'].lower()

# recipeid_instruc= data set of recipeId and its cooking instructions

In [ ]:
with open("C:/Users/varad/Desktop/CGAS_project/Ingre_cat.csv", newline='') as csvfile:
    now = csv.DictReader(csvfile)
    ingre_cat={}
    for row in now:
        ingre_cat[row['new_name'].lower()]= row['Dietrx_Category'].lower()

## ingre_cat= dataset with ingrediat name vs its corresponding 'Dietrx_Category'

In [ ]:
with open("C:/Users/varad/Desktop/CGAS_project/Recipe_small.csv", newline='', encoding='utf-8') as csvfile:
    now = csv.DictReader(csvfile)
    recipeid_name={}
    for row in now:
        recipeid_name[int(row['Recipe_id'])]= row['Recipe_title']

#recipeid_name= dataset with recipe Id vs Recipe name

In [ ]:
with open("C:/Users/varad/Desktop/CGAS_project/recipe_ingre.csv", newline='') as csvfile:
    now = csv.DictReader(csvfile)
    recipeid_ingre={}
    for row in now:
        idno=int(row['recipe_no'])
        ingre=row['ingredient'].lower()
        if(idno in recipeid_ingre):
            recipeid_ingre[idno].append(ingre)
        else:
            recipeid_ingre[idno]=[]
            recipeid_ingre[idno].append(ingre)
            
#recipeid_ingre= dataset of recipe Id vs set of ingrediants used

### ingre_categories is the list all categories present

In [ ]:
ingre_categories=[]

for i in ingre_cat:
    now=ingre_cat[i]
    if now not in ingre_categories:
        ingre_categories.append(now)


In [ ]:
print(sorted(ingre_categories))

['', 'additive', 'additive-salt', 'additive-sugar', 'additive-vinegar', 'additive-yeast', 'bakery', 'beverage', 'beverage-alcoholic', 'cereal', 'condiment', 'dairy', 'dish', 'essential oil', 'fish', 'flower', 'fruit', 'fungi', 'herb', 'legume', 'maize', 'meat', 'misc-other', 'misc-utensil', 'nuts and seeds', 'plant', 'plant derivative', 'seafood', 'spice', 'vegetable']


In [ ]:
sign_assg={'':'a','additive':'b','additive-salt':'c','additive-sugar':'d','additive-vinegar':'e','additive-yeast':'f','bakery':'g','beverage':'h','beverage-alcoholic':'i','cereal':'j','condiment':'k','dairy':'l','dish':'m','essential oil':'n','fish':'o','flower':'p','fruit':'q','fungi':'r','herb':'s','legume':'t','maize':'u','meat':'v','misc-other':'w','misc-utensil':'x','nuts and seeds':'y','plant':'z','plant derivative':'%','seafood':'$','spice':'&','vegetable':'!'}

### symbols are assigned to each category as below

In [ ]:
print(sign_assg)

{'': 'a', 'additive': 'b', 'additive-salt': 'c', 'additive-sugar': 'd', 'additive-vinegar': 'e', 'additive-yeast': 'f', 'bakery': 'g', 'beverage': 'h', 'beverage-alcoholic': 'i', 'cereal': 'j', 'condiment': 'k', 'dairy': 'l', 'dish': 'm', 'essential oil': 'n', 'fish': 'o', 'flower': 'p', 'fruit': 'q', 'fungi': 'r', 'herb': 's', 'legume': 't', 'maize': 'u', 'meat': 'v', 'misc-other': 'w', 'misc-utensil': 'x', 'nuts and seeds': 'y', 'plant': 'z', 'plant derivative': '%', 'seafood': '$', 'spice': '&', 'vegetable': '!'}


#### The two fnctions below are used for generating sequences for each recipe

In [ ]:
def final_seq(ings,ingre_cat,sign_assg):
    out=''
    for i in ings:
        #con=0
        now=''
        if(i in ingre_cat):
            now=sign_assg[ingre_cat[i]]
        else:
            now='a'
        
        out=out+now
    return(out)

In [ ]:
def make_seq(ing_seq,ins_seq,ingre_cat,sign_assg):
    count_ing=[]
    for i in ing_seq:
        count_ing.append([ins_seq.find(i),i])
    count_ing.sort()
    now=[]
    for i in count_ing:
        now.append(i[1])
    return(final_seq(now,ingre_cat,sign_assg))

In [ ]:
###-------------------------------------------

#### recipeid_seq is the dataset with recipe id and its corresponding sequence generated

In [ ]:
recipeid_seq={}
for i in recipeid_ingre:
    now=recipeid_ingre[i]
    if i in recipeid_instruc:
        noww=make_seq(now,recipeid_instruc[i],ingre_cat,sign_assg)
    else:
        noww=final_seq(now,ingre_cat,sign_assg)
    recipeid_seq[i]=noww

In [ ]:
len(recipeid_seq)

118112

#### Example of some of the sequences produced and its corresponding id

In [ ]:
dd=0
for i in recipeid_seq:
    dd+=1
    if(dd<11):
        print(i,recipeid_seq[i])

2610 &t!aa!!s&cs
2611 a&%!!sv&z
2612 v&%!ka!
2613 !%%tluus&sck
2614 &scyj&
2615 gdyyyqhll
2616 vj%jl!lhcy
2617 &j!mmac&%ys$z
2618 t&!!%&&h!
2619 tjmje&!ts!&aa%s!lv


In [ ]:
#B L RAMA KRISHNA_18BIT0175
## Blast-----------------

#### "score" function is used for scoring two sequences based on their best local allignment

In [ ]:
### Simple match and miss match is cnsidered for scoring in Blast
def score(a_que,b_data,scor,query,data,w,match,mis,T):
    left=a_que
    right=a_que+(w-1)
    left_data=b_data
    right_data=b_data+(w-1)
    best=-1
    while((left-1)>=0 & (left_data-1)>=0):
        left-=1
        left_data-=1
        if(query[left]==data[left_data]):
            scor+=match
            if(scor>best):
                best=scor
        elif(query[left]!=data[left_data]):
            scor+=mis
            if (scor<T):
                scor-=mis
                break
    
    while (((right+1)<len(query)) & ((right_data+1)<len(data))):
        right+=1
        right_data +=1
        if(query[right]==data[right_data]):
            scor+=match
            if(scor>best):
                best=scor
        elif(query[right]!=data[right_data]):
            scor+=mis
            if (scor<T):
                scor-=mis
                break
    return (left,right,left_data,right_data,scor,best)

#### The next 2 functions are used for generating Binary Search Tree''s to minimize time complexity while searching in Blast

In [ ]:
# Making binary search tree of the data sequence
def make_bst(a,memo):
    if (len(a)<=0):
        return None
    else:
        now=len(a)//2
        root=Node(a[now],memo[a[now]])
        root.left=make_bst(a[:now],memo)
        root.right=make_bst(a[now+1:],memo)
        return (root)


In [ ]:
def search_bst(a,r):
    if(r==None):
        return (None)
    elif (r.val==a):
        return (r.keys)
    elif(r.val>a):
        return (search_bst(a,r.left))
    else:
        return (search_bst(a,r.right))

#### Main Blast function take one sequence and the whole dataset of recipeid vs its sequence generated,
#### outputs the its best alignment score. Corresponding K-mer length[w], Threshold score(T), match score(match), miss match score(miss) 

In [ ]:
#VaradaRaghunathSaiPranav_2018418
def blastt(query,data,w,T,match,miss):
    w_list=[]
    for i in range(len(query)-w+1):
        w_list.append(query[i:i+w])
    
    dic_val_data={}

    for i in range(len(data)-w+1):
        temp=hash(data[i:i+w])
        if(temp in dic_val_data):
            dic_val_data[temp].append(i)
        else:
            dic_val_data[temp]=[i]

    dic_val_data_keys=list(dic_val_data.keys())
    dic_val_data_keys.sort()
    
    data_bst=make_bst(dic_val_data_keys,dic_val_data)
    
    update_final=[]
    for i in range(len(query)-w+1):
        temp=hash(query[i:i+w])
        anss=search_bst(temp,data_bst)
        if(anss is not None):
            for j in anss:
                now=list(score(i,j,w*match,query,data,w,match,miss,T))
                if((now not in update_final) & (now[-1]>=T)):
                    update_final.append(now[-1])         
    return(update_final)

In [ ]:
class Node:
    def __init__(self,val,keys):
        self.left=None
        self.right=None
        self.val=val
        self.keys=keys

In [ ]:
def first_do(a,data,w,T,match,miss):
    out={}
    for i in data:
        k=blastt(a,data[i],w,T,match,miss)
        if(len(k)>0):
            out[i]=max(k)
    return(out)

#### Define the variables below to customize the search.
#### Default demo values are already set

In [ ]:
input_sequence_id=18773
kmer_length=5
Treshold_score=6
match_match_score=1
miss_match_score=-1

In [ ]:
#first_do(input_sequence,data of sequences,kmer length=w,Treshold=T,match,miss)
out=first_do(recipeid_seq[input_sequence_id],recipeid_seq,kmer_length,Treshold_score,match_match_score,miss_match_score)
ppp=[]
for i in out:
    ppp.append([out[i],i])
ppp.sort()
print("Total No of alignments with Treshold as 'T'=",len(ppp))

Total No of alignments with Treshold as 'T'= 34


#### Top n matchs, n being user defined.
#### Note: The first top match will be of itself

In [ ]:
n=10
for i in ppp[::-1]:
    if(n>0):
        print("Score="+str(i[0]),recipeid_name[i[1]],i[1])
        n-=1

Score=12 Lunchbox Cookies 18773
Score=8 Sinful Brown Butter Nutella Stuffed Chocolate Chip Cookies 143707
Score=7 Chocolate Fudge Cookies With Toffee & Dried Cherries 143170
Score=7 Rhubarb Muffins 142641
Score=7 Mint Irish Cream Cookies 136197
Score=7 St. Patrick's Shamrock Sugar Cookies 134766
Score=7 Hazelnut Chocolate Chip Cookies 122587
Score=7 Pumpkin Polenta Cake 121331
Score=7 Cinnamon Shortcakes 102982
Score=7 Earl Grey Tea Cookies 102463
